In [1]:
import warnings
warnings.filterwarnings("ignore")
from collections import defaultdict
from tqdm.autonotebook import tqdm
import mysql.connector
import codecs
codecs.register(lambda name: codecs.lookup('utf8') if name == 'utf8mb4' else None)
import re

In [2]:
db = mysql.connector.connect(
  host="localhost",
  user="root",
  passwd='password',
  database='pruned_commits_db2',
  charset='utf8mb4')

cur = db.cursor()

In [4]:
cur.execute('SELECT lemma from lemmas;')
lemmas = [word[0] for word in cur.fetchall()]

In [24]:
_end = '_end_'
def make_trie(words):
    root = dict()
    for word in words:
        current_dict = root
        for letter in word:
            current_dict = current_dict.setdefault(letter, {})
        current_dict[_end] = _end
    return root

In [25]:
trie = make_trie(lemmas)

In [26]:
print(trie, file=open('trie.txt', 'w'))

In [27]:
len(trie)

41

In [34]:
from gensim.models import KeyedVectors

In [37]:
word_vectors = KeyedVectors.load_word2vec_format('/home/dkbrz/git_commits/by_month_vecs/2018-11.txt')

In [59]:
dictionary = dict()
with open('shorten_candidates.txt', 'w') as f:
    for word in lemmas:
        current_dict = trie
        candidates = []
        for key, letter in enumerate(word):
            current_dict = current_dict[letter]
            if _end in current_dict and word in word_vectors and word[:key+1] in word_vectors:
                cand = word[:key+1]
                if word_vectors.similarity(word, cand) >= 0.2 and len(cand)>1:
                    candidates.append(cand)
        if len(candidates) > 1:
            dictionary[word] = candidates[:-1]
            print(word+'\t'+';'.join(candidates), file=f)

In [60]:
reversed_dict = defaultdict(list)
for word in dictionary:
    for candidate in dictionary[word]:
        reversed_dict[candidate].append(word)

In [62]:
with open('shorten_candidates.txt', 'w') as f:
    for word in reversed_dict:
        print(word+'\t'+';'.join(reversed_dict[word]), file=f)